In [27]:
### Need to add points allowed (to do this, probabily need to look into matchups)

import nba_api
import pandas as pd
import sklearn
import os
import numpy as np
import requests
import time
import random
from sklearn.model_selection import KFold
import datetime as dt
import seaborn as sns
from nba_api.stats.static import players
import matplotlib.pyplot as plt
import math
from datetime import datetime, timedelta
from nba_api.stats.library.parameters import SeasonAllNullable
from nba_api.stats.library.parameters import SeasonAll
from nba_api.stats.library.parameters import SeasonTypeNullable
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
from tqdm import tqdm
from scipy.stats import norm
from nba_api.stats import endpoints
from nba_api.stats.static import teams
from sklearn.ensemble import RandomForestClassifier
import ipynb
import importlib

## Also read in ELO functions and tracked data

ELO_track = pd.read_csv('/Users/nickbachelder/Buckets Model/ELO_track', index_col=0)
ELO_track['Date'] = pd.to_datetime(ELO_track['Date'])

from ipynb.fs.defs.ELO_Tracking import get_pre_elo


## Also read in cleaned NBA data
NBAgamesC = pd.read_csv('/Users/nickbachelder/Buckets Model/NBAgamesC', index_col=0)
NBAgamesC['GAME_DATE_H'] = pd.to_datetime(NBAgamesC['GAME_DATE_H'])
nba_team = pd.read_csv('/Users/nickbachelder/Buckets Model/nba_team', index_col=0)
NBAgames = pd.read_csv('/Users/nickbachelder/Buckets Model/NBAgames', index_col=0)
NBAgames['GAME_DATE'] = pd.to_datetime(NBAgames['GAME_DATE'])

## Also import roster data

from ipynb.fs.defs.Roster_Injury_Implementation import nested_dict_folder
unfolded_roster = pd.read_csv('/Users/nickbachelder/Buckets Model/unfolded_roster', index_col=0)
folded_roster = nested_dict_folder(unfolded_roster, nba_team)

## Also import out data
from ipynb.fs.defs.Roster_Injury_Implementation import get_curr_injuries
from ipynb.fs.defs.Roster_Injury_Implementation import nested_dict_folder
unfolded_out = pd.read_csv('/Users/nickbachelder/Buckets Model/unfolded_out', index_col=0)
folded_out = nested_dict_folder(unfolded_out, nba_team)

# Also read in all box data for each player since 2015
all_boxes = pd.read_csv('/Users/nickbachelder/Buckets Model/all_boxes', index_col=0)
all_boxes['GAME_DATE'] = pd.to_datetime(all_boxes['GAME_DATE'])
NBAgamesC['GAME_ID_H'] = NBAgamesC['GAME_ID_H'].astype(int)

100%|██████████| 22/22 [00:00<00:00, 175.99it/s]


In [3]:
# create game_date column in all_boxes

def get_game_date(game_id):
    game = NBAgamesC[NBAgamesC['GAME_ID_H'] == game_id]
    date = game['GAME_DATE_H'].reset_index(drop = 1)[0]
    return(date)

all_boxes['GAME_DATE'] = [get_game_date(all_boxes.iloc[row,]["GAME_ID"]) for row in tqdm(range(len(all_boxes)))]

100%|██████████| 553777/553777 [04:35<00:00, 2006.58it/s]


In [28]:
pd.DataFrame.to_csv(all_boxes, '/Users/nickbachelder/Buckets Model/all_boxes')

In [44]:
def get_player_games_before(num_games, date, player_id, all_boxes):
    date = pd.to_datetime(date)
    player_box = all_boxes.loc[all_boxes['PLAYER_ID'] == player_id] 
    player_box_before = player_box.loc[player_box['GAME_DATE'] < date].sort_values('GAME_DATE', 
                                                                               ascending = False).reset_index(drop = 1).iloc[:num_games]
    return(player_box_before)
    
get_player_games_before(num_games = 15, date = '2021-08-15', player_id = 2544, all_boxes = all_boxes)

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,GAME_ID,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,REB,AST,STL,BLK,PTS,SEASON_YEAR,GAME_DATE
0,2544,LeBron James,1610612747,LAL,42000156,41.400000,11.0,26.0,0.423,3.0,10.0,0.300,4.0,6.0,0.667,9.0,7.0,2.0,2.0,29.0,2020-21,2021-06-03
1,2544,LeBron James,1610612747,LAL,42000155,31.643333,9.0,19.0,0.474,6.0,10.0,0.600,0.0,0.0,0.000,5.0,7.0,0.0,0.0,24.0,2020-21,2021-06-01
2,2544,LeBron James,1610612747,LAL,42000154,37.821667,10.0,21.0,0.476,1.0,7.0,0.143,4.0,7.0,0.571,12.0,6.0,1.0,0.0,25.0,2020-21,2021-05-30
3,2544,LeBron James,1610612747,LAL,42000153,37.753333,9.0,19.0,0.474,1.0,5.0,0.200,2.0,3.0,0.667,6.0,9.0,1.0,0.0,21.0,2020-21,2021-05-27
4,2544,LeBron James,1610612747,LAL,42000152,38.891667,9.0,16.0,0.563,4.0,9.0,0.444,1.0,1.0,1.000,4.0,9.0,2.0,0.0,23.0,2020-21,2021-05-25
5,2544,LeBron James,1610612747,LAL,42000151,36.061667,6.0,13.0,0.462,3.0,7.0,0.429,3.0,6.0,0.500,7.0,10.0,3.0,0.0,18.0,2020-21,2021-05-23
6,2544,LeBron James,1610612747,LAL,22001072,27.466667,11.0,22.0,0.500,2.0,5.0,0.400,1.0,2.0,0.500,1.0,6.0,3.0,0.0,25.0,2020-21,2021-05-16
7,2544,LeBron James,1610612747,LAL,22001061,28.440000,11.0,22.0,0.500,1.0,4.0,0.250,1.0,2.0,0.500,7.0,8.0,0.0,0.0,24.0,2020-21,2021-05-15
8,2544,LeBron James,1610612747,LAL,22000974,27.550000,6.0,11.0,0.545,2.0,5.0,0.400,5.0,7.0,0.714,7.0,6.0,0.0,0.0,19.0,2020-21,2021-05-02
9,2544,LeBron James,1610612747,LAL,22000956,31.850000,6.0,12.0,0.500,1.0,5.0,0.200,3.0,5.0,0.600,8.0,7.0,2.0,1.0,16.0,2020-21,2021-04-30


In [30]:
def get_player_playoff_min(date, player_id, all_boxes, NBAgamesC):
    date = pd.to_datetime(date)
    all_playoff_games = NBAgamesC[NBAgamesC['PLAYOFF_H'] == 1]
    all_playoff_games_unique = all_playoff_games['GAME_ID_H'].unique()
    all_player_games = all_boxes[all_boxes['PLAYER_ID'] == player_id]
    all_player_games = all_player_games[all_player_games['GAME_DATE'] < date]
    all_player_playoff_games = all_player_games[all_player_games['GAME_ID'].isin(all_playoff_games_unique)]
    all_playoff_min = all_player_playoff_games['MIN'].sum()
    
    return(all_playoff_min)


def get_team_playoff_games_past_5_years(teamabb, date, NBAgamesC, current_season):
    team_playoff_games = NBAgamesC[(NBAgamesC['PLAYOFF_H'] == 1)]
    team_playoff_games = team_playoff_games[(team_playoff_games['TEAM_ABBREVIATION_H'] == teamabb) | (team_playoff_games['TEAM_ABBREVIATION_A'] == teamabb)]
    date = pd.to_datetime(date)
    season = int(NBAgamesC[NBAgamesC['GAME_DATE_H'] <= date].reset_index(drop = 1).iloc[0,]['SEASON_ID_H'])
    if NBAgamesC[NBAgamesC['GAME_DATE_H'] > date].empty:
                 season = current_season
    past_season = int(season) - 1
    past_5_seasons = [past_season, past_season - 1, past_season - 2, past_season -3, past_season -4]
    team_playoff_games_past_5 = team_playoff_games[team_playoff_games['SEASON_ID_H'].isin(past_5_seasons)]
    return(len(team_playoff_games_past_5))
    
    
get_player_playoff_min('2016-06-03', 2544, all_boxes, NBAgamesC)
get_team_playoff_games_past_5_years('GSW', '2021-08-05', NBAgamesC, current_season = 2021)

60

In [31]:
def get_player_avgs_over_last_x_games(num_games, date, player_id, all_boxes):
    date = pd.to_datetime(date)
    player_box_before = get_player_games_before(num_games = num_games, date = date, 
                                                player_id = player_id, all_boxes = all_boxes)
    try:
        player_name = player_box_before['PLAYER_NAME'].unique()[0]
    except:
        final = pd.DataFrame({'player': np.repeat(0, 12), 'var': ['FG_PCT', 'FGA', 'FG3_PCT', 'FG3A', 'FT_PCT', 
                      'FTA', 'REB', 'AST', 'STL', 'BLK', 'PTS', 'MIN']}).T.reset_index(drop = 1)
        final.columns = final.iloc[1,]
        final = final.drop(1, axis = 0)
        final['PLAYER_NAME'] = 'G-League'
        return(final)
    player_box_before = player_box_before[['FG_PCT', 'FGA', 'FG3_PCT', 'FG3A', 'FT_PCT', 
                      'FTA', 'REB', 'AST', 'STL', 'BLK', 'PTS', 'MIN']]
    player_box_avgs = list(player_box_before.mean(axis = 0))
    player_box_avgs_cols = list(player_box_before.columns)
    final = pd.DataFrame({'player': player_box_avgs, 'var': player_box_avgs_cols}).T.reset_index(drop = 1)
    final.columns = final.iloc[1,]
    final = final.drop(1, axis = 0)
    final['PLAYER_NAME'] = player_name
    
    return(final)
    
    
get_player_avgs_over_last_x_games(num_games = 15, date = '2012-05-15', player_id = 2544, all_boxes = all_boxes)

1,FG_PCT,FGA,FG3_PCT,FG3A,FT_PCT,FTA,REB,AST,STL,BLK,PTS,MIN,PLAYER_NAME
0,0.509333,19.8667,0.3866,3.33333,0.754667,10,7.46667,5.13333,1.6,0.666667,29,37.2284,LeBron James


In [32]:
## write function to average player stats to make up team stats. In order to do this we need to model to match 
## previous game stats to game stats

In [33]:
def get_team_avgs_over_last_x_games(team_abb, date, num_games_for_avg, folded_roster, NBAgamesC,
                                    all_boxes, nba_team, current_season): 
    
    teamname = nba_team.loc[nba_team['abbreviation'] == team_abb]['full_name'].values[0]
    teamid = nba_team.loc[nba_team['abbreviation'] == team_abb]['id'].values[0]
    date = pd.to_datetime(date)
    season = int(NBAgamesC[NBAgamesC['GAME_DATE_H'] <= date].reset_index(drop = 1).iloc[0,]['SEASON_ID_H'])
    if NBAgamesC[NBAgamesC['GAME_DATE_H'] > date].empty:
                 season = current_season
    roster = folded_roster.get(season).get(teamid)
    
    team_stats = pd.DataFrame()
    for playerid in roster['PLAYER_ID']:
        playerstat = get_player_avgs_over_last_x_games(num_games = num_games_for_avg, date = date, 
                                                       player_id = playerid, all_boxes = all_boxes)
        playerstat['ALL_TIME_PLAYOFF_MIN'] = get_player_playoff_min(date, playerid, all_boxes, NBAgamesC)
        team_stats = pd.concat([team_stats, playerstat])
    team_stats = team_stats.fillna(0)
    return(team_stats)

        
get_team_avgs_over_last_x_games('GSW', date = '2021-09-15', num_games_for_avg = 15, 
                                folded_roster = folded_roster, NBAgamesC = NBAgamesC,
                                all_boxes = all_boxes, nba_team = nba_team, current_season = 2021)

1,FG_PCT,FGA,FG3_PCT,FG3A,FT_PCT,FTA,REB,AST,STL,BLK,PTS,MIN,PLAYER_NAME,ALL_TIME_PLAYOFF_MIN
0,0.400000,1.400000,0.166667,0.600000,0.050000,0.266667,1.200000,0.266667,0.466667,0.066667,2.266667,4.357444,Gary Payton II,0.000000
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,G-League,0.000000
0,0.556533,4.466667,0.438933,2.533333,0.333333,0.600000,2.666667,0.800000,0.800000,0.133333,6.733333,18.851444,Damion Lee,0.000000
0,0.275533,3.466667,0.137067,1.666667,0.133333,0.400000,0.866667,2.266667,0.266667,0.200000,3.266667,8.861000,Chris Chiozza,83.490000
0,0.407000,10.933333,0.350000,6.666667,0.701067,3.533333,2.666667,1.866667,0.600000,0.333333,14.800000,22.562889,Jordan Poole,0.000000
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,G-League,0.000000
0,0.557733,4.133333,0.000000,0.133333,0.316667,1.200000,8.000000,2.133333,0.133333,0.333333,5.066667,26.519000,Kevon Looney,818.431667
0,0.272667,4.333333,0.220000,2.533333,0.238867,1.000000,2.333333,1.466667,0.666667,0.266667,5.466667,13.811333,Nemanja Bjelica,77.306667
0,0.406600,2.733333,0.233333,1.666667,0.100000,0.266667,2.666667,2.200000,0.600000,0.600000,3.066667,19.516556,Andre Iguodala,5207.775000
0,0.457800,18.266667,0.438400,8.133333,0.577800,2.066667,4.600000,2.333333,1.600000,0.466667,22.066667,40.147556,Klay Thompson,4569.393333


In [34]:
## create function to get team injuries by folded_out
## finish

def get_out_players_for_game(team_id, game_id, folded_out, NBAgamesC):
    game = NBAgamesC.loc[NBAgamesC['GAME_ID_H'] == game_id]
    season = int(game['SEASON_ID_H'])
    out = folded_out.get(season).get(team_id)
    out_game = out[out['game_id'] == game_id].reset_index(drop = 1)['out'][0]
    
    return(eval(out_game))
    

get_out_players_for_game(team_id = 1610612753, game_id = 1521900077, folded_out =  folded_out, NBAgamesC = NBAgamesC)

['Aaron Gordon',
 'Jonathan Isaac',
 'Al-Farouq Aminu',
 'Mo Bamba',
 'Michael Carter-Williams',
 'Nikola Vucevic',
 'Evan Fournier',
 'James Ennis III',
 'Gary Clark',
 'B.J. Johnson',
 'D.J. Augustin',
 'Markelle Fultz',
 'Vic Law',
 'Khem Birch',
 'Wes Iwundu',
 'Terrence Ross',
 'Melvin Frazier']

In [35]:
def create_team_predictions(team_abb, date, num_games_for_avg, ELO_track, all_boxes, nba_team, NBAgamesC, folded_out, folded_roster, 
                            current_season, upcoming = False):
    
    team_id = nba_team[nba_team['abbreviation'] == team_abb]['id'].reset_index(drop =1)[0]
    team_stats = get_team_avgs_over_last_x_games(team_abb, date = date, num_games_for_avg = num_games_for_avg, 
                                folded_roster = folded_roster, NBAgamesC = NBAgamesC, 
                                                 all_boxes = all_boxes, nba_team = nba_team,
                                                current_season = current_season)
    
    ## account for injuries
    if upcoming == False:
        date = pd.to_datetime(date)
        games_before_date = NBAgamesC[NBAgamesC['GAME_DATE_H'] <= date]
        game_before_date_team = games_before_date[(games_before_date['TEAM_ABBREVIATION_H'] == team_abb) | 
                                                   (games_before_date['TEAM_ABBREVIATION_A'] == team_abb)].iloc[0,]
        game_id = game_before_date_team['GAME_ID_H']
        out = get_out_players_for_game(team_id = team_id, game_id = game_id, folded_out =  folded_out, 
                                       NBAgamesC = NBAgamesC)
    else:
        games_before_date = NBAgamesC[NBAgamesC['GAME_DATE_H'] <= date]
        game_before_date_team = games_before_date[(games_before_date['TEAM_ABBREVIATION_H'] == team_abb) | 
                                                   (games_before_date['TEAM_ABBREVIATION_A'] == team_abb)].iloc[0,]
        season = current_season
        out_all = get_curr_injuries()
        team_roster = list(folded_roster.get(season).get(team_id)['PLAYER'].reset_index(drop = 1))
        out = list(set.intersection(set(out_all), set(team_roster)))
    
    team_stats = team_stats[~(team_stats['PLAYER_NAME'].isin(out))] # update team stats to dislude injured players

    ## need to account for players stepping up when players are out . NOTE: percentages probs wont change
    
    min_sum = team_stats['MIN'].sum()
    min_in_game = 48*5
    min_diff = min_in_game - min_sum
    fix_time_amount = min_diff / len(team_stats)
    new_min_proj = team_stats['MIN'] + fix_time_amount
    proportion_all_player_time = new_min_proj / min_in_game
    proportion_orig_time = list(new_min_proj / team_stats['MIN'])
    stays_same = team_stats[['FG_PCT', 'FG3_PCT', 'FT_PCT', 'PLAYER_NAME']]
    
    playoff_min = team_stats[['ALL_TIME_PLAYOFF_MIN']].mul(proportion_all_player_time, axis=0)
    
    team_stats = team_stats.drop(['PLAYER_NAME', 'FG_PCT', 
                                  'FG3_PCT', 'FT_PCT', 'ALL_TIME_PLAYOFF_MIN'], 
                                 axis = 1).mul(proportion_orig_time, axis=0)
    
    team_stats = pd.concat([team_stats, stays_same, playoff_min], axis=1)
    
    ## take weighted averages
    fga = team_stats['FGA'].sum()
    fg3a = team_stats['FG3A'].sum()
    fta = team_stats['FTA'].sum()
    reb = team_stats['REB'].sum()
    ast = team_stats['AST'].sum()
    stl = team_stats['STL'].sum()
    blk = team_stats['BLK'].sum()
    fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
    fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
    ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
    tplayoff_min = team_stats['ALL_TIME_PLAYOFF_MIN'].sum()
    elo = get_pre_elo(team_abb, date, NBAgamesC = NBAgamesC, ELO_track = ELO_track)
    pts = team_stats['PTS'].sum()
    
    ## calculate p_win over last games, avg_margin over last games, and avg_points_allowed over last games
    games_before = NBAgamesC[NBAgamesC['GAME_DATE_H'] < date]
    games_before = games_before[(games_before['TEAM_ABBREVIATION_H'] == team_abb) | 
                                                   (games_before['TEAM_ABBREVIATION_A'] == team_abb)][0:num_games_for_avg]
    wins = 0
    margin_of_victory = []
    points_allowed = []
    for row in range(len(games_before)):
        game = games_before.iloc[row,]
        if game['TEAM_ABBREVIATION_H'] == team_abb:
            wins += game['WL_H']
            margin_of_victory.append(game['WINMARGIN_H'])
            points_allowed.append(game['PTS_A'])
        else:
            if game['WL_H'] == 0:
                wins += 1  
            margin_of_victory.append(-game['WINMARGIN_H'])
            points_allowed.append(game['PTS_H'])
    p_win = wins / num_games_for_avg
    margin_of_victory = sum(margin_of_victory) / num_games_for_avg
    avg_points_allowed = sum(points_allowed) / num_games_for_avg
    
    predict_row = pd.DataFrame({'var' : ['FGA', 'FG3A', 'FTA', 'REB', 'AST', 'STL', 'BLK', 
                                         'FG_P', 'FG_P3', 'FT_P', 'ELO', 'PTS', 'AVG_PLAYOFF_MIN', 'P_WIN', 'AVG_MARGIN', 'AVG_PTS_ALLOWED'],
                               'value' : [fga, fg3a, fta, reb, ast, stl, blk, fg_p, 
                                          fg_p3, ft_p, elo, pts, tplayoff_min, p_win, margin_of_victory, avg_points_allowed]}).T.reset_index(drop =1)
    
    predict_row.columns = predict_row.iloc[0,]
    predict_row= predict_row.drop(0, axis = 0)
    

    
    ### NEED TO account for teams playing less hard/more hard at end of season based on playoff needs
    
    ### NEED TO account for playoff experience
    
    
    
    return(predict_row)
    
create_team_predictions('GSW', '2021-08-12', num_games_for_avg = 10, folded_roster = folded_roster, 
                        all_boxes = all_boxes, folded_out = folded_out, NBAgamesC = NBAgamesC, nba_team = nba_team, 
                        ELO_track = ELO_track, upcoming = False, current_season = 2021)

,FGA,FG3A,FTA,REB,AST,STL,BLK,FG_P,FG_P3,FT_P,ELO,PTS,AVG_PLAYOFF_MIN,P_WIN,AVG_MARGIN,AVG_PTS_ALLOWED
1,88.7948,42.18,21.6597,43.1613,32.197,7.40856,4.69086,0.453311,0.341621,0.691816,1543,118.903,1983.4,0.8,10.8,107.8


In [41]:
def pregame_stat_2(team_h_abb, team_a_abb, gamedate, NBAgamesC, NBAgames, nba_team, 
                                  ELO_track, all_boxes, folded_out, folded_roster, 
                   num_games_for_avg, current_season, upcoming = False, playoff = 0):
    if playoff == 1: ## if playoff game, use last 35 games instead since players play more consistantly
        home_pred = create_team_predictions(team_h_abb, gamedate, num_games_for_avg = num_games_for_avg, folded_roster = folded_roster, 
                        all_boxes = all_boxes, folded_out = folded_out, NBAgamesC = NBAgamesC, nba_team = nba_team, 
                                            ELO_track = ELO_track, upcoming = False, current_season = current_season)
        away_pred = create_team_predictions(team_a_abb, gamedate, num_games_for_avg = num_games_for_avg,  NBAgamesC = NBAgamesC,
                                        folded_roster = folded_roster, folded_out = folded_out, all_boxes = all_boxes, nba_team = nba_team, 
                                        ELO_track = ELO_track, upcoming = False, current_season = current_season)
    else:
        home_pred = create_team_predictions(team_h_abb, gamedate, num_games_for_avg = num_games_for_avg, folded_roster = folded_roster, 
                        all_boxes = all_boxes, folded_out = folded_out, NBAgamesC = NBAgamesC, 
                                            nba_team = nba_team, ELO_track = ELO_track, upcoming = False, current_season = current_season)
        away_pred = create_team_predictions(team_a_abb, gamedate, num_games_for_avg = num_games_for_avg,  NBAgamesC = NBAgamesC,
                                        folded_roster = folded_roster, folded_out = folded_out, all_boxes = all_boxes, nba_team = nba_team, 
                                        ELO_track = ELO_track, upcoming = False, current_season = current_season)
        
    home_pred.columns = [str(col) + '_H' for col in home_pred.columns]
    
    away_pred.columns = [str(col) + '_A' for col in away_pred.columns]
    
    ## calculate away win percentages (highly correlated for playoff interaction)
    
    season = int(NBAgamesC[NBAgamesC['GAME_DATE_H'] <= gamedate].reset_index(drop = 1).iloc[0,]['SEASON_ID_H'])
    if NBAgamesC[NBAgamesC['GAME_DATE_H'] > gamedate].empty:
                 season = current_season
    a_win_p_A = NBAgamesC[(NBAgamesC['GAME_DATE_H'] < gamedate) & (NBAgamesC['TEAM_ABBREVIATION_A'] == team_a_abb) & (NBAgamesC['SEASON_ID_H'] == season)]
    a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
    away_pred['ROAD_WIN_P_A'] = a_win_p_A
    
    
    combined = pd.DataFrame(pd.concat([home_pred.reset_index(drop=True), away_pred.reset_index(drop=True)], axis = 1))
    
    ## add binary playoff variable
    if playoff == 1:
        combined['PLAYOFF'] = 1
    else:
        combined['PLAYOFF'] = 0
        
    ## add number of playoff games played by both teams in past 5
    combined['TEAM_PLAYOFF_GAMES_P5_H'] = get_team_playoff_games_past_5_years(team_h_abb, gamedate, NBAgamesC ,current_season)
    combined['TEAM_PLAYOFF_GAMES_P5_A'] = get_team_playoff_games_past_5_years(team_a_abb, gamedate, NBAgamesC,current_season)
    
    

    return(combined)
    
pregame_stat_2('BOS', 'GSW', '2021-09-12', NBAgamesC = NBAgamesC, NBAgames = NBAgames, nba_team = nba_team, 
                                  ELO_track = ELO_track, num_games_for_avg = 5, folded_out = folded_out, 
               all_boxes = all_boxes, folded_roster = folded_roster, playoff = 0, current_season = 2021)

<ipython-input-41-951703b07e82>:29: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)


,FGA_H,FG3A_H,FTA_H,REB_H,AST_H,STL_H,BLK_H,FG_P_H,FG_P3_H,FT_P_H,ELO_H,PTS_H,AVG_PLAYOFF_MIN_H,P_WIN_H,AVG_MARGIN_H,AVG_PTS_ALLOWED_H,FGA_A,FG3A_A,FTA_A,REB_A,AST_A,STL_A,BLK_A,FG_P_A,FG_P3_A,FT_P_A,ELO_A,PTS_A,AVG_PLAYOFF_MIN_A,P_WIN_A,AVG_MARGIN_A,AVG_PTS_ALLOWED_A,ROAD_WIN_P_A,PLAYOFF,TEAM_PLAYOFF_GAMES_P5_H,TEAM_PLAYOFF_GAMES_P5_A
0,84.0932,31.5222,30.393,33.4698,20.3242,5.38723,5.40074,0.402732,0.314698,0.671207,1487,110.529,1035.27,0.2,-11.2,123.4,91.404,42.3742,24.1185,45.9933,30.8229,8.27215,2.79775,0.474881,0.351216,0.779826,1543,126.412,1986.47,1,12.6,110.4,NaN,0,68,60


In [37]:
### start tomorrow by doing exploratory analysis on player minutes: 
## starter vs non-starter variability
## end of season decline in time for starters in good playoff standing

def create_predictdf_2(sample, NBAgamesC, NBAgames, nba_team, ELO_track, num_games_for_avg, 
                       all_boxes, folded_roster, folded_out, current_season, upcoming = False): ## sample is subset of NBAgamesC dataframe
    predictdf = pd.DataFrame()
    sample = sample.reset_index(drop = 1)
    
    for index in tqdm(range(len(sample))):
        
        team_h_abb = sample.iloc[index,:][['MATCHUP_H']][0][0:3]
        team_a_abb = sample.iloc[index,:][['MATCHUP_H']][0][-3:]
        gamedate = sample.iloc[index,:][['GAME_DATE_H']][0]
        playoff = sample.iloc[index,:][['PLAYOFF_H']][0]
    
        predictrow = pregame_stat_2(team_h_abb, team_a_abb, gamedate, NBAgamesC = NBAgamesC, NBAgames = NBAgames, nba_team = nba_team, 
                                  ELO_track = ELO_track, all_boxes = all_boxes, num_games_for_avg = num_games_for_avg, 
                                    folded_roster = folded_roster, folded_out = folded_out, playoff = playoff, current_season = current_season)
    
        predictdf = pd.concat([predictdf, predictrow])
     
    return predictdf

create_predictdf_2(sample = NBAgamesC[14540:14541], NBAgames = NBAgames, 
                               NBAgamesC = NBAgamesC, nba_team = nba_team, ELO_track = ELO_track, 
                               all_boxes = all_boxes, num_games_for_avg = 10, folded_roster = folded_roster, 
                   folded_out = folded_out, current_season = 2021)

100%|██████████| 1/1 [00:00<00:00,  3.33it/s]


,FGA_H,FG3A_H,FTA_H,REB_H,AST_H,STL_H,BLK_H,FG_P_H,FG_P3_H,FT_P_H,ELO_H,PTS_H,AVG_PLAYOFF_MIN_H,P_WIN_H,AVG_MARGIN_H,AVG_PTS_ALLOWED_H,FGA_A,FG3A_A,FTA_A,REB_A,AST_A,STL_A,BLK_A,FG_P_A,FG_P3_A,FT_P_A,ELO_A,PTS_A,AVG_PLAYOFF_MIN_A,P_WIN_A,AVG_MARGIN_A,AVG_PTS_ALLOWED_A,ROAD_WIN_P_A,PLAYOFF,TEAM_PLAYOFF_GAMES_P5_H,TEAM_PLAYOFF_GAMES_P5_A
0,86.4073,21.6213,26.8879,32.9223,22.8251,7.04353,3.66906,0.422631,0.298396,0.582512,1344,105.763,1977.34,0.4,-3.4,94.2,77.8376,19.56,32.9215,39.9055,20.8898,6.32663,6.67164,0.402517,0.276641,0.726845,1614,100.205,168.38,0.7,3.1,88.5,1.0,0,55,6


In [38]:
NBAgamesC

,SEASON_ID_H,TEAM_ID_H,TEAM_ABBREVIATION_H,TEAM_NAME_H,GAME_ID_H,GAME_DATE_H,MATCHUP_H,WL_H,MIN_H,PTS_H,FGM_H,FGA_H,FG_PCT_H,FG3M_H,FG3A_H,FG3_PCT_H,FTM_H,FTA_H,FT_PCT_H,OREB_H,DREB_H,REB_H,AST_H,STL_H,BLK_H,TOV_H,PF_H,PLAYOFF_H,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLAYOFF_A,WINMARGIN_H
0,2020,1610612749,MIL,Milwaukee Bucks,42000406,2021-07-20,MIL vs. PHX,1.0,240.0,105.0,37.0,82.0,0.451,6.0,27.0,0.222,25.0,29.0,0.862,11.0,42.0,53.0,20.0,10.0,6.0,18.0,17.0,1.0,1610612756,PHX,Phoenix Suns,PHX @ MIL,0.0,239.0,98.0,38.0,86.0,0.442,6.0,25.0,0.240,16.0,19.0,0.842,6.0,31.0,37.0,14.0,11.0,4.0,14.0,21.0,1.0,7.0
1,2020,1610612756,PHX,Phoenix Suns,42000405,2021-07-17,PHX vs. MIL,0.0,241.0,119.0,48.0,87.0,0.552,13.0,19.0,0.684,10.0,11.0,0.909,8.0,27.0,35.0,23.0,9.0,5.0,8.0,20.0,1.0,1610612749,MIL,Milwaukee Bucks,MIL @ PHX,1.0,240.0,123.0,50.0,87.0,0.575,14.0,28.0,0.500,9.0,17.0,0.529,11.0,26.0,37.0,26.0,7.0,1.0,11.0,17.0,1.0,-4.0
2,2020,1610612749,MIL,Milwaukee Bucks,42000404,2021-07-14,MIL vs. PHX,1.0,240.0,109.0,39.0,97.0,0.402,7.0,29.0,0.241,24.0,29.0,0.828,17.0,31.0,48.0,22.0,11.0,3.0,5.0,18.0,1.0,1610612756,PHX,Phoenix Suns,PHX @ MIL,0.0,240.0,103.0,40.0,78.0,0.513,7.0,23.0,0.304,16.0,19.0,0.842,5.0,35.0,40.0,18.0,3.0,9.0,17.0,24.0,1.0,6.0
3,2020,1610612749,MIL,Milwaukee Bucks,42000403,2021-07-11,MIL vs. PHX,1.0,240.0,120.0,43.0,90.0,0.478,14.0,36.0,0.389,20.0,26.0,0.769,13.0,34.0,47.0,28.0,8.0,2.0,9.0,18.0,1.0,1610612756,PHX,Phoenix Suns,PHX @ MIL,0.0,239.0,100.0,40.0,83.0,0.482,9.0,31.0,0.290,11.0,16.0,0.688,6.0,30.0,36.0,21.0,3.0,3.0,14.0,24.0,1.0,20.0
4,2020,1610612756,PHX,Phoenix Suns,42000402,2021-07-08,PHX vs. MIL,1.0,240.0,118.0,43.0,88.0,0.489,20.0,40.0,0.500,12.0,14.0,0.857,11.0,32.0,43.0,26.0,7.0,2.0,12.0,19.0,1.0,1610612749,MIL,Milwaukee Bucks,MIL @ PHX,0.0,240.0,108.0,42.0,93.0,0.452,9.0,31.0,0.290,15.0,23.0,0.652,18.0,28.0,46.0,21.0,8.0,7.0,9.0,17.0,1.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27734,2000,1610612762,UTA,Utah Jazz,20000010,2000-10-31,UTA vs. LAC,1.0,239.0,107.0,43.0,85.0,0.506,5.0,10.0,0.500,16.0,31.0,0.516,11.0,22.0,33.0,33.0,11.0,4.0,12.0,30.0,0.0,1610612746,LAC,Los Angeles Clippers,LAC @ UTA,0.0,240.0,94.0,34.0,69.0,0.493,4.0,13.0,0.308,22.0,34.0,0.647,12.0,32.0,44.0,18.0,3.0,6.0,24.0,32.0,0.0,13.0
27735,2000,1610612744,GSW,Golden State Warriors,20000011,2000-10-31,GSW vs. PHX,1.0,239.0,96.0,32.0,79.0,0.405,4.0,8.0,0.500,28.0,38.0,0.737,14.0,41.0,55.0,18.0,11.0,5.0,21.0,22.0,0.0,1610612756,PHX,Phoenix Suns,PHX @ GSW,0.0,240.0,94.0,36.0,91.0,0.396,6.0,21.0,0.286,16.0,20.0,0.800,11.0,33.0,44.0,25.0,12.0,3.0,16.0,28.0,0.0,2.0
27736,2000,1610612757,POR,Portland Trail Blazers,20000012,2000-10-31,POR vs. LAL,0.0,239.0,86.0,34.0,85.0,0.400,4.0,11.0,0.364,14.0,16.0,0.875,13.0,19.0,32.0,18.0,13.0,1.0,10.0,28.0,0.0,1610612747,LAL,Los Angeles Lakers,LAL @ POR,1.0,239.0,96.0,36.0,63.0,0.571,5.0,11.0,0.455,19.0,30.0,0.633,8.0,31.0,39.0,28.0,5.0,8.0,20.0,18.0,0.0,-10.0
27737,2000,1610612751,BKN,Brooklyn Nets,20000002,2000-10-31,BKN vs. CLE,0.0,239.0,82.0,31.0,85.0,0.365,3.0,10.0,0.300,17.0,26.0,0.654,12.0,35.0,47.0,24.0,9.0,8.0,12.0,31.0,0.0,1610612739,CLE,Cleveland Cavaliers,CLE @ BKN,1.0,240.0,86.0,32.0,78.0,0.410,2.0,7.0,0.286,20.0,32.0,0.625,11.0,41.0,52.0,16.0,5.0,8.0,19.0,27.0,0.0,-4.0


In [24]:
### find rows with na that had issues and filter NBAgamesC to those games to see what happened: they are preseason
import warnings
warnings.simplefilter("ignore", UserWarning)



## create massive training set that includes all games with complete averages possibily computed after 2015

predictdf_10 = create_predictdf_2(sample = NBAgamesC[:14541], NBAgames = NBAgames, 
                               NBAgamesC = NBAgamesC, nba_team = nba_team, ELO_track = ELO_track, 
                               all_boxes = all_boxes, num_games_for_avg = 10, folded_roster = folded_roster, folded_out = folded_out
                                , current_season = 2021)
predictdf_10 = predictdf_10.iloc[:14541,:].reset_index(drop = 1) ## Omit games before there was enough data to collect averages
predictdf_10['Results'] = pd.DataFrame(NBAgamesC.iloc[:14541,:].reset_index()['WL_H'])
results_10 = pd.DataFrame(predictdf_10['Results']).reset_index(drop = 1)
predictdf_10 = predictdf_10.drop('Results', axis = 1).reset_index(drop = 1)


predictGames_10 = NBAgamesC[:14541][~predictdf_10.isnull().any(axis=1)]

  8%|▊         | 1106/14541 [05:03<1:00:23,  3.71it/s]<ipython-input-18-187fb4b4aca5>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
  8%|▊         | 1117/14541 [05:06<1:00:36,  3.69it/s]<ipython-input-18-187fb4b4aca5>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
  8%|▊         | 1118/14541 [05:06<1:00:49,  3.68it/s]<ipython-input-18-187fb4b4aca5>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
  8%|▊         | 1121/14541 [05:07<1:01:05,  3.66it/s]<ipython-input-18-187fb4b4aca5>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
  8%|▊         | 1123/14541 [05:08<1:00:57,  3.67it/s]<ipython-input-18-187fb4b4aca5>:24: RuntimeWarning

<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
 16%|█▌        | 2312/14541 [10:41<58:45,  3.47it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Runt

 16%|█▌        | 2319/14541 [10:43<56:26,  3.61it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Runt

<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
 16%|█▌        | 2327/14541 [10:46<56:51,  3.58it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Runt

 16%|█▌        | 2334/14541 [10:48<56:01,  3.63it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Runt

...

<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-18-187fb4b4aca5>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
 84%|████████▎ | 12151/14541 [53:30<10:02,  3.97it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: i

 84%|████████▎ | 12158/14541 [53:31<10:02,  3.95it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Run

<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
 84%|████████▎ | 12166/14541 [53:33<09:59,  3.96it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Run

<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-18-187fb4b4aca5>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
 84%|████████▎ | 12173/14541 [53:35<09:56,  3.97it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: i

NameError: name 'results' is not defined

In [25]:
predictdf_fixed_10 = predictdf_10[~predictdf_10.isnull().any(axis=1)]
results_fixed_10 = results_10[~predictdf_10.isnull().any(axis=1)]

pd.DataFrame.to_csv(results_fixed_10, '/Users/nickbachelder/Buckets Model/results_10')
pd.DataFrame.to_csv(predictdf_fixed_10, '/Users/nickbachelder/Buckets Model/predictdf_10')
pd.DataFrame.to_csv(predictGames_10, '/Users/nickbachelder/Buckets Model/training_games_10')

,FGA_H,FG3A_H,FTA_H,REB_H,AST_H,STL_H,BLK_H,FG_P_H,FG_P3_H,FT_P_H,ELO_H,PTS_H,AVG_PLAYOFF_MIN_H,P_WIN_H,AVG_MARGIN_H,AVG_PTS_ALLOWED_H,FGA_A,FG3A_A,FTA_A,REB_A,AST_A,STL_A,BLK_A,FG_P_A,FG_P3_A,FT_P_A,ELO_A,PTS_A,AVG_PLAYOFF_MIN_A,P_WIN_A,AVG_MARGIN_A,AVG_PTS_ALLOWED_A,ROAD_WIN_P_A,PLAYOFF,TEAM_PLAYOFF_GAMES_P5_H,TEAM_PLAYOFF_GAMES_P5_A
0,92.4409,31.6908,24.0246,48.6086,25.8234,7.54777,4.61556,0.474787,0.294488,0.571335,1698,118.194,1923.42,0.685714,4.08571,110.943,86.3477,31.8569,18.6874,42.0468,24.9573,6.61145,4.57989,0.480997,0.362017,0.62594,1700,112.904,1482.59,0.657143,4.31429,108.486,0.652174,1,38,0
1,86.5214,32.6913,18.8646,41.3152,24.9529,6.62939,4.55748,0.48483,0.361409,0.642326,1709,113.786,1509.42,0.685714,4.51429,108.2,92.6588,31.9592,23.881,48.8445,25.982,7.60855,4.77067,0.473698,0.291889,0.569246,1689,117.971,1889.03,0.685714,5.05714,110.229,0.531915,1,0,38
2,92.6939,32.2241,23.6985,49.0074,26.0893,7.56365,4.86894,0.479134,0.296404,0.568869,1683,119.23,1855.31,0.685714,5.08571,110.629,86.5447,32.8107,18.8843,41.4697,25.1431,6.8899,4.45116,0.483686,0.365153,0.643682,1715,113.97,1469.45,0.714286,4.71429,108.714,0.666667,1,38,0
3,92.1443,34.1944,22.9208,47.8021,24.6624,7.34413,4.72652,0.469994,0.312573,0.545392,1669,118.655,1691.58,0.657143,4.48571,111.429,86.3008,32.5518,19.1873,42.2172,24.5722,7.06257,4.5898,0.476973,0.355859,0.615683,1729,113.362,1318.06,0.714286,4.54286,108.457,0.681818,1,38,0
4,86.3416,32.5819,19.2553,42.2789,24.6733,7.1251,4.70522,0.478368,0.354684,0.629568,1723,113.503,1276.75,0.714286,4.48571,108.629,93.656,35.2364,22.8829,47.9111,24.8102,7.10776,4.61022,0.476927,0.316542,0.567253,1675,121.479,1700.31,0.657143,4.4,111.714,0.543478,1,0,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14536,84.1423,14.4855,23.6406,41.763,11.4415,6.865,5.35455,0.382028,0.302801,0.494395,1361,94.634,145.93,0.5,-1.2,93.1,83.0464,12.0663,25.5982,48.0458,25.3361,6.30709,3.26463,0.453389,0.249176,0.554385,1349,99.0813,113.261,0.5,-3.3,94.3,1.000000,0,23,6
14537,83.0648,17.6497,18.577,35.8139,26.4748,7.56334,5.63631,0.477715,0.323281,0.496521,1399,103.467,311.942,0.3,-2.4,91.9,79.3524,11.7521,26.4841,41.4953,15.0154,5.06024,5.22394,0.388023,0.211635,0.629478,1603,90.4946,808.858,0.4,-4.5,92.8,0.750000,0,12,29
14538,71.5506,12.8426,23.7406,36.5959,25.222,4.91746,4.82872,0.441002,0.310717,0.649798,1526,90.9614,816.613,0.3,-7.2,89.6,84.3153,21.3581,28.5914,37.0045,24.4326,9.03406,5.45496,0.425068,0.300122,0.653274,1507,104.088,234.449,0.5,-1.4,94.6,0.600000,0,4,6
14539,81.9732,13.2538,25.6813,38.0798,20.2992,6.56942,3.8127,0.452524,0.30146,0.612231,1580,100.15,2941.83,0.3,-4,84.9,69.0773,26.8766,21.7559,32.89,13.5517,5.72593,6.24132,0.438723,0.377371,0.627537,1416,91.6661,464.041,0.5,-3.1,100.8,0.500000,0,64,0


In [31]:
NBAgamesC[14590:14591]

,SEASON_ID_H,TEAM_ID_H,TEAM_ABBREVIATION_H,TEAM_NAME_H,GAME_ID_H,GAME_DATE_H,MATCHUP_H,WL_H,MIN_H,PTS_H,FGM_H,FGA_H,FG_PCT_H,FG3M_H,FG3A_H,FG3_PCT_H,FTM_H,FTA_H,FT_PCT_H,OREB_H,DREB_H,REB_H,AST_H,STL_H,BLK_H,TOV_H,PF_H,PLAYOFF_H,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLAYOFF_A,WINMARGIN_H
14590,2010,1610612750,MIN,Minnesota Timberwolves,1521000032,2010-07-14,MIN vs. SAC,0.0,200.0,78.0,32.0,69.0,0.464,5.0,13.0,0.385,9.0,13.0,0.692,6.0,25.0,31.0,15.0,6.0,4.0,22.0,29.0,0.0,1610612758,SAC,Sacramento Kings,SAC @ MIN,1.0,201.0,79.0,31.0,65.0,0.477,4.0,11.0,0.364,13.0,21.0,0.619,9.0,29.0,38.0,15.0,7.0,4.0,25.0,22.0,0.0,-1.0


In [32]:
### find rows with na that had issues and filter NBAgamesC to those games to see what happened: they are preseason
import warnings
warnings.simplefilter("ignore", UserWarning)



## create massive training set that includes all games with complete averages possibily computed after 2015

predictdf_10_2 = create_predictdf_2(sample = NBAgamesC[:14541], NBAgames = NBAgames, 
                               NBAgamesC = NBAgamesC, nba_team = nba_team, ELO_track = ELO_track, 
                               all_boxes = all_boxes, num_games_for_avg = 10, folded_roster = folded_roster, folded_out = folded_out,
                                   current_season = 2021)
predictdf_10_2 = predictdf_10_2.iloc[:14541,:].reset_index(drop = 1) ## Omit games before there was enough data to collect averages
predictdf_10_2['Results'] = pd.DataFrame(NBAgamesC.iloc[:14541,:].reset_index()['WL_H'])
results_10_2 = pd.DataFrame(predictdf_10_2['Results']).reset_index(drop = 1)
predictdf_10_2 = predictdf_10_2.drop('Results', axis = 1).reset_index(drop = 1)


predictGames_10_2 = NBAgamesC[:14541][~predictdf_10_2.isnull().any(axis=1)]

  8%|▊         | 1106/14541 [05:47<1:02:58,  3.56it/s]<ipython-input-30-c05e6e34a40e>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
  8%|▊         | 1117/14541 [05:50<1:02:46,  3.56it/s]<ipython-input-30-c05e6e34a40e>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
  8%|▊         | 1118/14541 [05:50<1:02:55,  3.56it/s]<ipython-input-30-c05e6e34a40e>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
  8%|▊         | 1121/14541 [05:51<1:02:53,  3.56it/s]<ipython-input-30-c05e6e34a40e>:24: RuntimeWarning: invalid value encountered in double_scalars
  a_win_p_A = (len(a_win_p_A) - a_win_p_A['WL_H'].sum()) / len(a_win_p_A)
  8%|▊         | 1123/14541 [05:52<1:02:46,  3.56it/s]<ipython-input-30-c05e6e34a40e>:24: RuntimeWarning

<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
 16%|█▌        | 2312/14541 [12:00<1:13:10,  2.79it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Ru

 16%|█▌        | 2319/14541 [12:02<1:07:15,  3.03it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Ru

<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
 16%|█▌        | 2327/14541 [12:04<1:05:35,  3.10it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Ru

 16%|█▌        | 2334/14541 [12:07<1:03:26,  3.21it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: Ru

...

 83%|████████▎ | 12136/14541 [1:00:15<10:55,  3.67it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: R

 84%|████████▎ | 12143/14541 [1:00:17<10:42,  3.73it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: R

 84%|████████▎ | 12150/14541 [1:00:19<10:30,  3.79it/s]<ipython-input-16-915a02f96b29>:33: RuntimeWarning: divide by zero encountered in double_scalars
  fix_time_amount = min_diff / len(team_stats)
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered

 84%|████████▎ | 12157/14541 [1:00:21<10:20,  3.84it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: R

 84%|████████▎ | 12171/14541 [1:00:25<10:10,  3.88it/s]<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: RuntimeWarning: invalid value encountered in double_scalars
  ft_p = (team_stats['FT_PCT'] * team_stats['FTA']).sum() / team_stats['FTA'].sum()
<ipython-input-16-915a02f96b29>:55: RuntimeWarning: invalid value encountered in double_scalars
  fg_p = (team_stats['FG_PCT'] * team_stats['FGA']).sum() / team_stats['FGA'].sum()
<ipython-input-16-915a02f96b29>:56: RuntimeWarning: invalid value encountered in double_scalars
  fg_p3 = (team_stats['FG3_PCT'] * team_stats['FG3A']).sum() / team_stats['FG3A'].sum()
<ipython-input-16-915a02f96b29>:57: R

In [34]:
predictdf_fixed_10_2 = predictdf_10_2[~predictdf_10_2.isnull().any(axis=1)]
results_fixed_10_2 = results_10_2[~predictdf_10_2.isnull().any(axis=1)]

pd.DataFrame.to_csv(results_fixed_10_2, '/Users/nickbachelder/Buckets Model/results_10_2')
pd.DataFrame.to_csv(predictdf_fixed_10_2, '/Users/nickbachelder/Buckets Model/predictdf_10_2')
pd.DataFrame.to_csv(predictGames_10_2, '/Users/nickbachelder/Buckets Model/training_games_10_2')